In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import csv


from keras.layers import Input, Dense
from keras.models import Model
from numpy import linalg as LA
import sklearn
from sklearn import metrics
from scipy.signal import find_peaks
from statistics import mean
from sklearn.cluster import KMeans

Using TensorFlow backend.


In [15]:
def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l

def ITD_mean_cal(l):
    x=mean(l)
    return x

def Refined_mean(l):
    ITD_collect=[]
    kmeans = KMeans(n_clusters=2, random_state=0).fit(l)
    X=kmeans.labels_
    #print(X)
    one_count = np.count_nonzero(X == 1)
    zero_count=np.count_nonzero(X == 0)
    #print("one_count=",one_count,",zero_count=",zero_count)
    if(one_count>=zero_count):
        index_list= np.where(X == 1)
        count=one_count
    else:
        index_list= np.where(X == 0)
        count=zero_count
    for i in range(count):
        #print(type(index_list[0][i].astype(np.int)))\
        ITD_collect.append(l[np.int16(index_list[0][i]).item()])
    #print(ITD_collect)
    ITD_collect=np.array(ITD_collect)
    mean=np.mean(ITD_collect)
    return mean
    
def Special_char_count(l): 
    no_splchar=0
    for keyAscii in l:
        if(not(keyAscii>=48 and keyAscii <= 57) and not(keyAscii >= 65 and keyAscii <= 90) and not(keyAscii >= 97 and keyAscii <= 122) and keyAscii!=-5):
            
            no_splchar+=1
            
    return no_splchar 

def backspce_count(l):
    no_backspce=0
    for keyAscii in l:
        if(keyAscii== -5):
            no_backspce+=1
    return no_backspce

usr_dir="Data_files/"
for user in range(1,23,1):
    usr_file=usr_dir+"U"+str(user)+"_LIHF_itd_file.csv"
    
    data=[]
    happy_bucket=[]
    sad_bucket=[]
    stressed_bucket=[]
    relax_bucket=[]
    #################
    happy_char=[]
    sad_char=[]
    stressed_char=[]
    relax_char=[]
    
    print(usr_file)
    dataset=pd.read_csv(usr_file,header=None)
    dataframe=dataset.values
    
    #print("row_number="+str(dataframe.shape[0]))
    
    # value list holds unique id 
    value_list=[]
    for i in range(dataframe.shape[0]):
        value_list.append(dataframe[i][0])
        
    ID_list=list(set(value_list))
    ID_list.sort()
    print("number of unique ID="+str(len(ID_list)))
    start=0
    next_start=0
    # traverse the whole list and add list of ITD values in emotion list
    for i in range(len(ID_list)):
        ID_number=ID_list[i]
        #print("ID_number="+str(ID_number))
        ITD_values=[]
        #### for time domain feature took character list #####
        char_list=[]
        for j in range(start,dataframe.shape[0],1):
            if(dataframe[j][0]==ID_number):
                ITD_values.append(dataframe[j][2])
                char_list.append(dataframe[j][3])
                next_start=next_start+1
            else:
                start=next_start
                #print(next_start)
                break
        if(dataframe[j-1][4]==2 and len(ITD_values)>3):
            happy_bucket.append(list(ITD_values))
            time_char_feature=[Special_char_count(char_list),backspce_count(char_list),len(char_list)]
            happy_char.append(list(time_char_feature))
            
            
        if(dataframe[j-1][4]==-2 and len(ITD_values)>3):
            sad_bucket.append(list(ITD_values))
            time_char_feature=[Special_char_count(char_list),backspce_count(char_list),len(char_list)]
            sad_char.append(list(time_char_feature))  
            
        if(dataframe[j-1][4]==1 and len(ITD_values)>3):
            stressed_bucket.append(list(ITD_values))
            time_char_feature=[Special_char_count(char_list),backspce_count(char_list),len(char_list)]
            stressed_char.append(list(time_char_feature)) 
            
        if(dataframe[j-1][4]==0 and len(ITD_values)>3):
            relax_bucket.append(list(ITD_values))
            time_char_feature=[Special_char_count(char_list),backspce_count(char_list),len(char_list)]
            relax_char.append(list(time_char_feature))
        #print("ID_number="+str(ID_number)+","+"length of list="+str(len(ITD_values)))
    print("sum of bucket length="+str(len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)))
    count_ITD_list=len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)
    
    print("Happy list length=",len(happy_bucket),"sad list length=",len(sad_bucket))
    print("happy char list length=",len(happy_char),"sad char list length=",len(sad_char))
    print("Stressed list length=",len(stressed_bucket),"relaxed list length=",len(relax_bucket))
    
    print("stressed char list length=",len(stressed_char),"Relaxed char list length=",len(relax_char))
    
    ############### raw data reading #############################
    
    #target_file="encoded_input/Amp/"+"U"+str(user)+"_LIHF_itd_file.csv"
    #target_data=pd.read_csv(target_file,header=None)
    #target_array=target_data.values
    
    #col_count=target_array.shape[1]
    #raw_data=target_array[:,0:col_count-1]
    #print("raw data size=",raw_data.shape[0])
    #######################################################
    #happy_index=0
    for (itd_list,char_list) in zip(happy_bucket,happy_char):
        # time feature calculation
        l=np.array(itd_list)
        l=np.reshape(l,(len(l),1))
        time_list=[ITD_mean_cal(itd_list),Refined_mean(l)]
        ###########################
        peak_list=[]
        freq_list=[]
        gap_list=[]
        signal=np.array(itd_list,dtype=float)
        # taking absolute value after fft
        sp=np.abs(np.fft.fft(signal))
        # frequency from fft
        freq=np.fft.fftfreq(sp.size)
        # indicies for peaks
        peaks,_=find_peaks(sp)
        for i in range(len(peaks)):
            peak_list.append(sp[peaks[i]])
            freq_list.append(freq[peaks[i]])
            
        peak_list=np.array(peak_list)
        freq_list=np.array(freq_list)
        #ind = np.argpartition(peak_list, -3)[-3:]
        #print(peak_list)
        
        # find indices of top most amplitude in sorted order
        ind=peak_list.argsort()[-3:][::-1]
        #print(peak_list[ind])
        #print(freq_list[ind])
        # concatenate peak amplitude and frequency
        #print(peak_list[ind])
        for val in ind:
            gap_list.append(peaks[val])
        gap_list.sort()
        
        #print(gap_list)
        
        for k in range(len(gap_list)-1):
            gap_list[k]=gap_list[k+1]-gap_list[k]
        if(len(gap_list)!=0):
            gap_list.remove(gap_list[-1])
        #gap_list.append(2)
        #print(freq[ind])
        row=np.concatenate((peak_list[ind],freq_list[ind]))
        # whole data is collected in data
        temp=np.append(row,gap_list).tolist()
        if(len(temp)<8):
            temp=pad(temp,0,8)
        temp.append(2)  
        temp=time_list+char_list+temp
        data.append(temp)
        
    
    sad_index=0    
    for (itd_list,char_list) in zip(sad_bucket,sad_char):
        l=np.array(itd_list)
        l=np.reshape(l,(len(l),1))
        time_list=[ITD_mean_cal(itd_list),Refined_mean(l)]
        
        peak_list=[]
        freq_list=[]
        gap_list=[]
        signal=np.array(itd_list,dtype=float)
        sp=np.abs(np.fft.fft(signal))
        freq=np.fft.fftfreq(sp.size)
        peaks,_=find_peaks(sp)
        for i in range(len(peaks)):
            peak_list.append(sp[peaks[i]])
            freq_list.append(freq[peaks[i]])
        peak_list=np.array(peak_list)
        freq_list=np.array(freq_list)
        #ind = np.argpartition(peak_list, -3)[-3:]
        #print(peak_list)
        ind=peak_list.argsort()[-3:][::-1]
        
        for val in ind:
            gap_list.append(peaks[val])
        gap_list.sort()  
        #print(gap_list)
        for k in range(len(gap_list)-1):
            gap_list[k]=gap_list[k+1]-gap_list[k]
        if(len(gap_list)!=0):
            gap_list.remove(gap_list[-1])
        #gap_list.append(-2)
        #print(peak_list[ind])
        #print(freq[ind])
        
        row=np.concatenate((peak_list[ind],freq_list[ind]))
        # whole data is collected in data
        temp=np.append(row,gap_list).tolist()
        if(len(temp)<8):
            temp=pad(temp,0,8)
        temp.append(-2)
        temp=time_list+char_list+temp
        data.append(temp)
        
     
 
    for (itd_list,char_list) in zip(stressed_bucket,stressed_char):
        l=np.array(itd_list)
        l=np.reshape(l,(len(l),1))
        time_list=[ITD_mean_cal(itd_list),Refined_mean(l)]
        
        peak_list=[]
        freq_list=[]
        gap_list=[]
        signal=np.array(itd_list,dtype=float)
        sp=np.abs(np.fft.fft(signal))
        freq=np.fft.fftfreq(sp.size)
        peaks,_=find_peaks(sp)
        for i in range(len(peaks)):
            peak_list.append(sp[peaks[i]])
            freq_list.append(freq[peaks[i]])
        peak_list=np.array(peak_list)
        freq_list=np.array(freq_list)
        #ind = np.argpartition(peak_list, -3)[-3:]
        #print(peak_list)
        ind=peak_list.argsort()[-3:][::-1]
        for val in ind:
            gap_list.append(peaks[val])
        gap_list.sort()  
        #print(gap_list)
        for k in range(len(gap_list)-1):
            gap_list[k]=gap_list[k+1]-gap_list[k]
        if(len(gap_list)!=0):
            
            gap_list.remove(gap_list[-1])
        #gap_list.append(1)
        #print(peak_list[ind])
        #print(freq[ind])
        row=np.concatenate((peak_list[ind],freq_list[ind]))
        # whole data is collected in data
        temp=np.append(row,gap_list).tolist()
        if(len(temp)<8):
            temp=pad(temp,0,8)
        temp.append(1)   
        temp=time_list+char_list+temp
        data.append(temp)
        
    
      
    for (itd_list,char_list) in zip(relax_bucket,relax_char):
        l=np.array(itd_list)
        l=np.reshape(l,(len(l),1))
        time_list=[ITD_mean_cal(itd_list),Refined_mean(l)]
        
        peak_list=[]
        freq_list=[]
        gap_list=[]
        signal=np.array(itd_list,dtype=float)
        sp=np.abs(np.fft.fft(signal))
        freq=np.fft.fftfreq(sp.size)
        peaks,_=find_peaks(sp)
        for i in range(len(peaks)):
            peak_list.append(sp[peaks[i]])
            freq_list.append(freq[peaks[i]])
        peak_list=np.array(peak_list)
        freq_list=np.array(freq_list)
        #ind = np.argpartition(peak_list, -3)[-3:]
        #print(peak_list)
        ind=peak_list.argsort()[-3:][::-1]
        for val in ind:
            gap_list.append(peaks[val])
        gap_list.sort()  
        #print(gap_list)
        for k in range(len(gap_list)-1):
            gap_list[k]=gap_list[k+1]-gap_list[k]
        if(len(gap_list)!=0):
            
            gap_list.remove(gap_list[-1])
        #gap_list.append(0)
        #print(peak_list[ind])
        #print(freq[ind])
        row=np.concatenate((peak_list[ind],freq_list[ind]))
        # whole data is collected in data
        temp=np.append(row,gap_list).tolist()
        if(len(temp)<8):
            temp=pad(temp,0,8)
        temp.append(0) 
        temp=time_list+char_list+temp
        data.append(temp)
        
        
    print("length of data=",len(data))
    
    for i in range(len(data)):
        if(len(data[i])<9):
            
            print(data[i])
            
    # this part for writting raw data with feature data        
    #raw_data=np.append(raw_data,data,axis=1)
    #print(raw_data)
    #np.savetxt("encoded_input/time_feature_augmented/"+"U"+str(user)+"_LIHF_itd_file.csv", raw_data, delimiter=",")
    
    ############################################
    writefile="encoded_input/FullTime_freq_feature/"+"U"+str(user)+"_LIHF_itd_file.csv"
    with open(writefile,"w") as resultfile:
            writer=csv.writer(resultfile)
            writer.writerows(data)

Data_files/U1_LIHF_itd_file.csv
number of unique ID=114
sum of bucket length=114
Happy list length= 31 sad list length= 23
happy char list length= 31 sad char list length= 23
Stressed list length= 9 relaxed list length= 51
stressed char list length= 9 Relaxed char list length= 51
length of data= 114
Data_files/U2_LIHF_itd_file.csv
number of unique ID=45
sum of bucket length=45
Happy list length= 13 sad list length= 8
happy char list length= 13 sad char list length= 8
Stressed list length= 4 relaxed list length= 20
stressed char list length= 4 Relaxed char list length= 20
length of data= 45
Data_files/U3_LIHF_itd_file.csv
number of unique ID=48
sum of bucket length=45
Happy list length= 5 sad list length= 7
happy char list length= 5 sad char list length= 7
Stressed list length= 2 relaxed list length= 31
stressed char list length= 2 Relaxed char list length= 31
length of data= 45
Data_files/U4_LIHF_itd_file.csv
number of unique ID=118
sum of bucket length=118
Happy list length= 5 sad lis